[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/protosome/convergent_overlapping_gene_prediction/blob/main/codon_overlap.ipynb)

In [ ]:
!git clone https://github.com/protosome/convergent_overlapping_gene_prediction.git

In [ ]:
%cd convergent_overlapping_gene_prediction/

In [ ]:
pip install biopython

In [ ]:
pip install tensorflow_addons

In [ ]:
#@title ###**Load dependencies**.

from google.colab import drive
import ipywidgets as widgets
from IPython.display import display
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import matplotlib.pyplot as plt
import pandas as pd
import csv
import json
import nltk
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from Bio.Seq import CodonTable
from tensorflow.keras.losses import Loss
import tensorflow_addons as tfa
import itertools as it
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
#@title ###**Build tokenizer**.

### FOR TOKENIZER, OPTIMALLY USE THE TEST DATASET SINCE IT IS SMALL
# to test the trained model
with open("/content/convergent_overlapping_gene_prediction/tokenizer_dataset/concat_pos_neg_114.json") as concat:
    concat_test_model = json.load(concat)

with open("/content/convergent_overlapping_gene_prediction/tokenizer_dataset/overlap_114.json") as overlap:
    overlap_test_model = json.load(overlap)

# Tokenization and text vectorization
max_length = 103 #note, this should be the max len of the overlap, not input sequences
vocab_size = 35


# We must use the same tokenizer dataset as that used to train the original model
def tokenize(sentences):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=vocab_size, filters='')
    tokenizer.fit_on_texts(sentences)
    return tokenizer

concat_tokenizer = tokenize(concat_test_model)
overlap_tokenizer = tokenize(overlap_test_model)

def vectorize(tokenizer, sentences):
    seqs = tokenizer.texts_to_sequences(sentences)
    return tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=max_length, padding='post')

concat_test_model_np = np.array(concat_test_model)
overlap_test_model_np = np.array(overlap_test_model)

concat_test_model_np_tokenizer = tokenize(concat_test_model_np)
overlap_test_model_np_tokenizer = tokenize(overlap_test_model_np)


In [ ]:
#@title ###**Load the overlap prediction model**.

model_to_predict = tf.keras.models.load_model('/content/convergent_overlapping_gene_prediction/convergent_overlap_prediction_model')


In [ ]:
#@title ###**Prediction functions**.

def predict_overlapping_sequence(trained_model, aa_sequences):

    def translate_to_dna_with_all_options(aa_sequence: str) -> list:


    # Codons and their frequencies for each amino acid based on the E. coli table
        back_translation_code_with_all_options = {
            'A': [('GCG', 0.27), ('GCT', 0.26), ('GCC', 0.26), ('GCA', 0.21)],
            'C': [('TGC', 0.53), ('TGT', 0.47)],
            'D': [('GAT', 0.63), ('GAC', 0.37)],
            'E': [('GAA', 0.68), ('GAG', 0.32)],
            'F': [('TTT', 0.58), ('TTC', 0.42)],
            'G': [('GGC', 0.35), ('GGT', 0.32), ('GGG', 0.25), ('GGA', 0.08)],
            'H': [('CAT', 0.56), ('CAC', 0.44)],
            'I': [('ATT', 0.48), ('ATC', 0.39), ('ATA', 0.14)],
            'K': [('AAA', 0.74), ('AAG', 0.26)],
            'L': [('CTG', 0.43), ('CTT', 0.13), ('CTC', 0.13), ('TTA', 0.14), ('CTA', 0.07), ('TTG', 0.13)],
            'M': [('ATG', 1.00)],
            'N': [('AAC', 0.60), ('AAT', 0.40)],
            'P': [('CCG', 0.52), ('CCA', 0.19), ('CCT', 0.16), ('CCC', 0.13)],
            'Q': [('CAG', 0.66), ('CAA', 0.34)],
            'R': [('CGT', 0.36), ('CGC', 0.36), ('CGG', 0.11), ('AGA', 0.08), ('AGG', 0.05), ('CGA', 0.04)],
            'S': [('AGC', 0.24), ('TCC', 0.24), ('TCT', 0.17), ('TCG', 0.15), ('TCA', 0.14), ('AGT', 0.15)],
            'T': [('ACC', 0.36), ('ACA', 0.28), ('ACG', 0.25), ('ACT', 0.11)],
            'V': [('GTG', 0.46), ('GTT', 0.28), ('GTC', 0.15), ('GTA', 0.11)],
            'W': [('TGG', 1.00)],
            'Y': [('TAT', 0.59), ('TAC', 0.41)],
            '*': [('TAA', 0.61), ('TGA', 0.30), ('TAG', 0.09)]
        }

        def choose_codon_based_on_frequency(codons):
            """
            Randomly choose a codon based on the frequency distribution.

            Args:
            - codons (list): A list of tuples where each tuple contains a codon and its frequency

            Returns:
            - list: A list containing the chosen codon
            """
            # Extract codon names and their frequencies
            codon_names = [codon for codon, _ in codons]
            codon_freqs = [freq for _, freq in codons]

            # Normalize the frequencies to ensure they sum up to 1
            total_frequency = sum(codon_freqs)
            normalized_freqs = [freq/total_frequency for freq in codon_freqs]

            # Randomly select a codon based on the frequency distribution
            chosen_codon = np.random.choice(codon_names, p = normalized_freqs)

            return [chosen_codon]

        # For each amino acid in the sequence, choose a codon based on its frequency
        list_of_list_of_codons = [choose_codon_based_on_frequency(back_translation_code_with_all_options[aa]) for aa in aa_sequence]

        # Combine the codons to form the nucleotide sequence
        list_of_combinations = [''.join(combination) for combination in it.product(*list_of_list_of_codons)]

        return list_of_combinations

    def translate(model, tokenizer, input_text):
        input_vectorized = vectorize(tokenizer, [input_text])
        prediction = model.predict(input_vectorized)[0]
        predicted_indices = np.argmax(prediction, axis=-1)
        #note that, we use the same tokenizer from the training data so that the output has the same tokens assigned
        predicted_text = overlap_tokenizer.sequences_to_texts([predicted_indices])[
            0].strip()
        return predicted_text


    def convert_chars_for_translated_overlap(string):
        string = string.replace("no overlap", "#", 1)  # Replace first occurrence of "no overlap" with "#"
        string = string.replace("overlap", "#", 1)  # Replace first occurrence of "overlap" with "#"
        string = string.replace("no", "#", 1)  # Replace first occurrence of "no" with "#"
        if string == "#":
            return string
        new_string = ""
        for char in string:
            if char.lower() == "b":
                new_string += "A"
            elif char.lower() == "j":
                new_string += "G"
            elif char.lower() == "o":
                new_string += "T"
            elif char.lower() == "u":
                new_string += "C"
            else:
                new_string += char
        return new_string.replace(" ", "")

    # Running the above sequence through the model, using the tokenizer run on the data from the original model training process
    translated_overlap = translate(trained_model, concat_test_model_np_tokenizer, aa_sequences)

    overlap_output_from_model = convert_chars_for_translated_overlap(translated_overlap)

    # Generate the rc to include in the next small df
    reverse_complement_overlap = Seq(overlap_output_from_model)
    reverse_complement_overlap = str(reverse_complement_overlap.reverse_complement())

    # Create a DataFrame with two rows: original and reversed sequences
    # Note, the rc sequence actually comes first here, followed by the model output overlap sequence
    output_forward_reverse = pd.DataFrame({"overlap_sequence": [reverse_complement_overlap, overlap_output_from_model]})

    # Splitting and collapsing the sequences
    sequence_list = aa_sequences.split()
    collapsed_sequence = ''.join(sequence_list)

    # Splitting the sequence at the asterisk and keeping the asterisk
    split_sequences_with_asterisk = [seq + '*' for seq in collapsed_sequence.split('*') if seq]

    # Creating a dataframe from the sequences
    df_sequences = pd.DataFrame(split_sequences_with_asterisk, columns=["amino_acid_sequence"])

    # Adding the back translated dna sequences to the data frame.
    df_sequences['nt_sequence'] = df_sequences['amino_acid_sequence'].apply(lambda aa_seq: translate_to_dna_with_all_options(aa_seq)[0])

    df_sequences["overlap_seq"] = output_forward_reverse

    #here, we are attaching the overlap sequence to the known coding sequence, generated above. It is
    #added at the location based on the length of the overlap_seq
    def modify_sequence(df):
        # Check if 'overlap_seq' column exists in the DataFrame
        if 'overlap_seq' not in df.columns:
            raise ValueError("DataFrame must contain 'overlap_seq' column.")

        # Calculate the length of the overlap sequence
        df['overlap_length'] = df['overlap_seq'].apply(len)

        # Modify the sequences
        df['modified_sequence'] = df.apply(lambda row: row['nt_sequence'][:-row['overlap_length']] + row['overlap_seq'], axis=1)

        return df

    modify_sequence(df_sequences)

    # Custom function to translate using Seq
    def translate_with_seq(sequence):
        coding_dna = Seq(sequence)
        return str(coding_dna.translate())

    # Apply the custom function to the 'nt_sequence' column
    df_sequences['translated_sequence'] = df_sequences['modified_sequence'].apply(translate_with_seq)

    #print(compare_aa)
    return(df_sequences)


def find_matching_sequence(model_to_predict, formatted_aa_seq, max_attempts, _counter=[0]):
    _counter[0] += 1
    print(f"Parsing row {_counter[0]}...")

    attempts = 0
    match_found = False
    matching_dataframe = None

    while attempts < max_attempts:
        attempts += 1

        try:
            result_df = predict_overlapping_sequence(model_to_predict, formatted_aa_seq)
        except CodonTable.TranslationError as e:
            print(f"Error: {e}")
             # Print the "overlap_length" at the end of each iteration
            print(f"Attempt {attempts}... Overlap Length: {matching_dataframe['overlap_length'].iloc[0] if matching_dataframe is not None else 'N/A'}")
            continue

        # Check if the "nt_sequence" and "translated_sequence" columns match in both rows
        if (result_df['amino_acid_sequence'].iloc[0] == result_df['translated_sequence'].iloc[0] and
            result_df['amino_acid_sequence'].iloc[1] == result_df['translated_sequence'].iloc[1]):
            match_found = True
            matching_dataframe = result_df
            break

    if match_found:
         # Print the "overlap_length" at the end of each iteration
        print(f"Attempt {attempts}... Overlap Length: {matching_dataframe['overlap_length'].iloc[0] if matching_dataframe is not None else 'N/A'}")
        return matching_dataframe
    else:
        print("There is no predicted significant overlap")
        return None


In [ ]:
#@title ###**Process the input amino acid sequences**. Note, both sequences must be at least 33 amino acids long

sequence_1 = "MLALAPVIAAVGEGRESMGGWKKIYRAALNGELEPMPGPDLPPPPPWSTVIHELWDASRTLPWTHLRGPLAPAMLRDHHDQALAVASAQPPG" #@param {type:"string"}
sequence_2 = "MIIWFVAPGTRVLLLPVVALRRGILQKAPMERAWWTATLVLVAMHATDLPLFDSRLNILGWTLLAGLAAFNQEVGQMPQPRPDRDGPAASPEPKAP" #@param {type:"string"}

def process_sequences(aa_seq_1, aa_seq_2):

    if len(aa_seq_1) < 33:
        return "Error: sequence_1 is not long enough to process. The minimum length is 33 amino acids."
    if len(aa_seq_2) < 33:
        return "Error: sequence_2 is not long enough to process. The minimum length is 33 amino acids."

    # Keep only the final 34 amino acids of each sequence
    aa_seq_1_trimmed = aa_seq_1[-34:]
    aa_seq_2_trimmed = aa_seq_2[-34:]

    # Replace the first amino acid with an 'M'
    aa_seq_1_processed = 'M' + aa_seq_1_trimmed[1:]
    aa_seq_2_processed = 'M' + aa_seq_2_trimmed[1:]

    # Add an asterisk after each sequence
    aa_seq_1_processed += '*'
    aa_seq_2_processed += '*'

    # Concatenate the two sequences
    concatenated_seq = aa_seq_1_processed + aa_seq_2_processed

    # Add a space between every character
    final_seq = ' '.join(concatenated_seq)

    # Return the processed sequence
    return final_seq

concatenated_sequence = process_sequences(sequence_1, sequence_2)

# Check if the result is an error message or the processed sequence, and print accordingly
if "Error" in concatenated_sequence:
    print(concatenated_sequence)
else:
    print("The input amino acid sequences have been processed and concatenated: ")
    print(concatenated_sequence)


In [ ]:
#@title ###**Predict a convergent overlapping sequence**.

formatted_aa_seq = concatenated_sequence

inference_rounds = "10" # @param ["10", "100", "200", "300", "500", "1000"] {type:"string"}

inference_rounds = int(inference_rounds)

testing_df = find_matching_sequence(model_to_predict, formatted_aa_seq, inference_rounds)

testing_df